In [1]:
import pandas as pd

In [2]:
train = pd.read_csv( "dataset/train.csv")
test = pd.read_csv("dataset/test.csv")

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
train = train[:100000]
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.096120,0.00999,0.053270,0.003140,0.049670,0.008540
std,0.294757,0.09945,0.224573,0.055948,0.217263,0.092017
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [5]:
classes = list(train.columns)[2:]
classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [6]:
train[classes].sum()

toxic            9612
severe_toxic      999
obscene          5327
threat            314
insult           4967
identity_hate     854
dtype: int64

In [7]:
links = '(http://.*?\s)|(http://.*)'
ip_addr = '\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'
users = '\[\[User.*'
newline = '\\n'
print(train['comment_text'].str.contains(links).sum())
print(train['comment_text'].str.contains(ip_addr).sum())
print(train['comment_text'].str.contains(users).sum())
print(train['comment_text'].str.contains(newline).sum())

C:\Users\admin\AppData\Local\Temp\ipykernel_30872\1688125639.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print(train['comment_text'].str.contains(links).sum())


2766
6313
196
59357


In [10]:
def clean(comment):
  import re
  comment=comment.lower()
  comment=re.sub(links,'',comment)
  comment=re.sub(ip_addr,'',comment)
  comment=re.sub(users,'',comment)
  comment=re.sub(newline,'',comment)
  return comment

In [11]:
train['comment_text']=train['comment_text'].map(lambda i:clean(i))
test['comment_text']=test['comment_text'].map(lambda i:clean(i))

In [12]:
print(train['comment_text'].str.contains(links).sum())
print(train['comment_text'].str.contains(ip_addr).sum())
print(train['comment_text'].str.contains(users).sum())
print(train['comment_text'].str.contains(newline).sum())

C:\Users\admin\AppData\Local\Temp\ipykernel_30872\1882578715.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print(train['comment_text'].str.contains(links).sum())


0
0
0
0


In [13]:
x=train['comment_text']
y=train.iloc[:,2:8]
print(x.shape)
print(y.shape)

(100000,)
(100000, 6)


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=1)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tfidf=TfidfVectorizer(max_features=5000,strip_accents='unicode',stop_words='english',token_pattern=r'\w{2,}')

In [17]:
tfidf.fit(X_train)
X_train_feat=tfidf.fit_transform(X_train)
X_train_feat.shape

(80000, 5000)

In [18]:
X_val_feat=tfidf.transform(X_val)
X_val_feat.shape

(20000, 5000)

In [20]:
import numpy as np

labels = pd.read_csv( "dataset/test_labels.csv")
labels=labels.iloc[:,1:]
sum_labels=np.sum(labels,axis=1)
idx=sum_labels>=0

In [21]:
labels_consider=labels[idx]
test=test.iloc[:,1:]
tests_consider=test[idx].values[:,0]

print(labels_consider.shape, tests_consider.shape)

(63978, 6) (63978,)


In [22]:
X_test_feat=tfidf.transform(tests_consider)
X_test_feat.shape

(63978, 5000)

In [24]:
!pip install scikit-multilearn
print(X_train.shape)
print(y_train.shape)

(80000,)
(80000, 6)



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [26]:

model_br = BinaryRelevance(classifier = LogisticRegression(C=20.0, max_iter=1000000), require_dense = [False, True])
model_br.fit(X_train_feat, y_train)


BinaryRelevance(classifier=LogisticRegression(C=20.0, max_iter=1000000),
                require_dense=[False, True])

In [24]:
## Binary Relevance

In [27]:
preds_train = model_br.predict(X_train_feat)

In [28]:
from sklearn.metrics import roc_auc_score, accuracy_score

print(roc_auc_score(y_train, preds_train.toarray()))

0.8168988057992655


In [29]:
preds_val = model_br.predict(X_val_feat)


In [30]:
print(roc_auc_score(y_val, preds_val.toarray()))

0.7252186237232792


In [31]:
preds_test = model_br.predict(X_test_feat)

In [32]:
print(roc_auc_score(labels_consider, preds_test.toarray()))

0.7321764236115941
